In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
import tqdm
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
now = dt.datetime.now()
X = 25.814000
Y = 35.373000

In [3]:
GEO_SPLIT = 1
BLOCK_SIZE = 64
START_TIME = now - dt.timedelta(days=300)
END_TIME = now
X_mid = X // GEO_SPLIT * GEO_SPLIT + GEO_SPLIT / 2
Y_mid = Y // GEO_SPLIT * GEO_SPLIT + GEO_SPLIT / 2
MIN_LAT = Y_mid - 5
MAX_LAT = Y_mid + 5
MIN_LON = X_mid - 5
MAX_LON = X_mid + 5

In [4]:
def make_params(starttime, endtime, minlatitude, maxlatitude, minlongitude, maxlongitude):
    params = {
        "format": "geojson",
        "starttime": starttime,
        "endtime": endtime,
        "minlatitude": minlatitude,
        "maxlatitude": maxlatitude,
        "minlongitude": minlongitude,
        "maxlongitude": maxlongitude
    }
    return params

In [5]:
def get_earthquake_count(params):
    url = "https://earthquake.usgs.gov/fdsnws/event/1/count"
    response = requests.get(url, params=params)
    return response.json()["count"]

In [6]:
def get_earthquake_data(params):
    url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
    response = requests.get(url, params=params)
    return response

In [7]:
def make_df(resp, params, errors):
    all_eqs = []
    try:
        for eq in resp.json()["features"]:
            prop = list(eq["properties"].values())
            prop.extend(eq["geometry"]["coordinates"])
            all_eqs.append(prop)
        cols = list(resp.json()["features"][0]["properties"].keys())
        cols.extend(["longitude", "latitude", "depth"])
        df = pd.DataFrame(all_eqs, columns=cols)
    except:
        errors.append(params)
        df = pd.DataFrame()
    return df, errors

In [8]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    return distance

In [9]:
def get_data(START_TIME, END_TIME, MIN_LAT, MAX_LAT, MIN_LON, MAX_LON):
    while True:
        print(START_TIME)
        count = get_earthquake_count(make_params(START_TIME, END_TIME, MIN_LAT, MAX_LAT, MIN_LON, MAX_LON))
        print(count)
        print("=====================================")
        if count < 65:
            START_TIME = START_TIME - dt.timedelta(days=300)
        else:
            resp = get_earthquake_data(make_params(START_TIME, END_TIME, MIN_LAT, MAX_LAT, MIN_LON, MAX_LON))
            df, errors = make_df(resp, make_params(START_TIME, END_TIME, MIN_LAT, MAX_LAT, MIN_LON, MAX_LON), [])
            df = df[df["type"] == "earthquake"]
            df = df[["time", "longitude", "latitude", "depth", "mag", "magType"]]
            df["time"] = df["time"].apply(lambda x: dt.datetime.fromtimestamp(x/1000))
            df.dropna(inplace=True)
            df.drop_duplicates(inplace=True)
            df["longitude_disc"] = (df["longitude"] // GEO_SPLIT * GEO_SPLIT).astype(int)
            df["latitude_disc"] = (df["latitude"] // GEO_SPLIT * GEO_SPLIT).astype(int)
            df["pos"] = "0_0"
            df.loc[
                (df["latitude"] >= Y - GEO_SPLIT / 2)
                & (df["latitude"] <= Y + GEO_SPLIT / 2)
                & (df["longitude"] >= X - GEO_SPLIT / 2)
                & (df["longitude"] <= X + GEO_SPLIT / 2),
                "pos",
            ] = (
                str(Y - GEO_SPLIT / 2) + "_" + str(X - GEO_SPLIT / 2)
            )
            df["distance"] = haversine_distance(df["latitude"], df["longitude"], Y, X)
            df["latitude_disc"] = Y - GEO_SPLIT / 2
            df["longitude_disc"] = X - GEO_SPLIT / 2
            tmp1 = df[df["pos"] != "0_0"]
            if len(tmp1) > 0:
                tmp2 = df[df["distance"] <= 300]
                tmp2 = tmp2[tmp2["time"] <= tmp1["time"].max()]
                print(len(tmp1), len(tmp2))
                if len(tmp2) >= BLOCK_SIZE + 1:
                    return df[df["time"] <= tmp1["time"].max()], START_TIME, errors
            START_TIME = START_TIME - dt.timedelta(days=300)

In [10]:
df, START_TIME, errors = get_data(START_TIME, END_TIME, MIN_LAT, MAX_LAT, MIN_LON, MAX_LON)

2023-03-09 13:12:00.011535


116
2 33
2022-05-13 13:12:00.011535
265
11 90


In [11]:
df

,time,longitude,latitude,depth,mag,magType,longitude_disc,latitude_disc,pos,distance
48,2023-08-29 00:10:19.406,25.4996,35.7724,11.932,4.5,mb,25.314,34.873,34.873_25.314,52.734474
49,2023-08-28 13:12:25.928,22.0863,38.2850,11.267,4.2,mb,25.314,34.873,0_0,463.514448
50,2023-08-27 05:09:43.189,27.8094,36.3227,67.601,4.4,mb,25.314,34.873,0_0,208.553122
51,2023-08-25 16:50:52.485,26.1494,34.1420,10.000,4.0,mb,25.314,34.873,0_0,140.268094
52,2023-08-21 17:45:57.264,28.1337,36.7359,73.187,4.3,mb,25.314,34.873,0_0,257.769675
...,...,...,...,...,...,...,...,...,...,...
260,2022-05-16 23:23:48.975,26.2954,33.9649,10.000,4.0,mb,25.314,34.873,0_0,162.644601
261,2022-05-15 23:49:56.094,28.5676,36.5108,57.360,4.0,mb,25.314,34.873,0_0,278.294859
262,2022-05-15 07:34:30.215,25.3417,36.1811,10.000,4.2,mb,25.314,34.873,0_0,99.446007
263,2022-05-14 11:25:02.555,25.5387,34.0408,10.600,4.1,mb,25.314,34.873,0_0,150.256019


In [12]:
df_tp = pd.read_csv("../data/all.csv")
df_tp.drop_duplicates(inplace=True)
df_tp

,plate,lat,lon
0,am,30.754,132.824
1,am,30.970,132.965
2,am,31.216,133.197
3,am,31.515,133.500
4,am,31.882,134.042
...,...,...,...
12314,yz,20.561,112.784
12315,yz,20.137,113.030
12316,yz,19.713,113.274
12317,yz,19.288,113.517


In [13]:
def find_min_dist(df_tp, x, y):
    df_tp["dist"] = haversine_distance(x, y, df_tp["lat"], df_tp["lon"])
    min_dist = df_tp["dist"].min()
    plates = df_tp[df_tp["dist"] == min_dist].sort_values("plate")["plate"].tolist()
    plates = "_".join(plates)
    return min_dist, plates

In [14]:
def add_region_info(df, df_tp):
    region2plate = {}
    region2dist = {}
    df["lat_cent"] = df["latitude_disc"] + 0.5
    df["lon_cent"] = df["longitude_disc"] + 0.5
    for pos in tqdm.tqdm(df["pos"].unique()):
        x, y = pos.split("_")
        x, y = float(x), float(y)
        dist, plate = find_min_dist(df_tp, x + 0.5, y + 0.5)
        region2plate[pos] = plate
        region2dist[pos] = dist
    df["plate_region"] = df["pos"].map(region2plate)
    df["dist_region"] = df["pos"].map(region2dist)
    return df

In [15]:
def add_tectonic_info(df, df_tp):
    coordinates = list(zip(df['latitude'], df['longitude']))
    results = list(tqdm.tqdm(map(lambda x: find_min_dist(df_tp, x[0], x[1]), coordinates), total=len(coordinates)))
    df[["dist", "plate"]] = results
    return df

In [16]:
df = add_region_info(df, df_tp)
df = add_tectonic_info(df, df_tp)

100%|██████████| 217/217 [00:00<00:00, 315.68it/s]


In [17]:
df

,time,longitude,latitude,depth,mag,magType,longitude_disc,latitude_disc,pos,distance,lat_cent,lon_cent,plate_region,dist_region,dist,plate
48,2023-08-29 00:10:19.406,25.4996,35.7724,11.932,4.5,mb,25.314,34.873,34.873_25.314,52.734474,35.373,25.814,AS_nu,126.842021,169.540935,AS_nu
49,2023-08-28 13:12:25.928,22.0863,38.2850,11.267,4.2,mb,25.314,34.873,0_0,463.514448,35.373,25.814,nu_sa,1420.841222,10.321476,AS_eu
50,2023-08-27 05:09:43.189,27.8094,36.3227,67.601,4.4,mb,25.314,34.873,0_0,208.553122,35.373,25.814,nu_sa,1420.841222,87.842779,AS_AT
51,2023-08-25 16:50:52.485,26.1494,34.1420,10.000,4.0,mb,25.314,34.873,0_0,140.268094,35.373,25.814,nu_sa,1420.841222,29.695357,AS_nu
52,2023-08-21 17:45:57.264,28.1337,36.7359,73.187,4.3,mb,25.314,34.873,0_0,257.769675,35.373,25.814,nu_sa,1420.841222,36.926865,AS_AT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,2022-05-16 23:23:48.975,26.2954,33.9649,10.000,4.0,mb,25.314,34.873,0_0,162.644601,35.373,25.814,nu_sa,1420.841222,30.262044,AS_nu
261,2022-05-15 23:49:56.094,28.5676,36.5108,57.360,4.0,mb,25.314,34.873,0_0,278.294859,35.373,25.814,nu_sa,1420.841222,25.879318,AS_AT
262,2022-05-15 07:34:30.215,25.3417,36.1811,10.000,4.2,mb,25.314,34.873,0_0,99.446007,35.373,25.814,nu_sa,1420.841222,215.709663,AS_nu
263,2022-05-14 11:25:02.555,25.5387,34.0408,10.600,4.1,mb,25.314,34.873,0_0,150.256019,35.373,25.814,nu_sa,1420.841222,23.417555,AS_nu


In [18]:
def map_col(df, col, mapping):
    mapping = dict(zip(mapping.iloc[:, 0], mapping.iloc[:, 1]))
    df[col] = df[col].map(mapping)
    return df

In [19]:
mapping1 = pd.read_csv("../data/magtype2id.csv")
mapping2 = pd.read_csv("../data/plate2id.csv")
mapping3 = pd.read_csv("../data/plate_region2id.csv")

In [20]:
df = map_col(df, "magType", mapping1)
df = map_col(df, "plate", mapping2)
df = map_col(df, "plate_region", mapping3)

In [21]:
df.isna().sum()

time              0
longitude         0
latitude          0
depth             0
mag               0
magType           0
longitude_disc    0
latitude_disc     0
pos               0
distance          0
lat_cent          0
lon_cent          0
plate_region      0
dist_region       0
dist              0
plate             0
dtype: int64

In [22]:
PREPROC_PARAMS = {
    "mag_low": -1,
    "mag_high": 7,
    "depth_low": 2,
    "depth_high": 1e8,
    "dist_low": 1,
    "dist_high": 1e8,
    "dist_region_low": 2,
    "dist_region_high": 1e8,
    "scale_distance": 78.44,
    "scale_distance_lag": 300,
}

In [23]:
def preprocess_df(df, PREPROC_PARAMS, scaler_dict):

    scaler = scaler_dict["mag"]
    df["mag"] = scaler.transform(np.clip(df["mag"].values, PREPROC_PARAMS["mag_low"], PREPROC_PARAMS["mag_high"]).reshape(-1, 1))

    scaler = scaler_dict["depth"]
    df["depth"] = np.log(df["depth"] + np.abs(df["depth"].min()) + 1)
    df["depth"] = scaler.transform(np.clip(df["depth"].values, PREPROC_PARAMS["depth_low"], PREPROC_PARAMS["depth_high"]).reshape(-1, 1))

    scaler = scaler_dict["latitude_new"]
    df["latitude_new"] = scaler.transform(df["latitude"].values.reshape(-1, 1))

    scaler = scaler_dict["longitude_new"]
    df["longitude_new"] = scaler.transform(df["longitude"].values.reshape(-1, 1))

    scaler = scaler_dict["lat_cent"]
    df["lat_cent"] = scaler.transform(df["lat_cent"].values.reshape(-1, 1))

    scaler = scaler_dict["lon_cent"]
    df["lon_cent"] = scaler.transform(df["lon_cent"].values.reshape(-1, 1))

    scaler = scaler_dict["dist"]
    df["dist"] = df["dist"].astype(float)
    df["dist"] = scaler.transform(np.clip(np.log(df["dist"] + 1).values.reshape(-1, 1), PREPROC_PARAMS["dist_low"], PREPROC_PARAMS["dist_high"]))

    scaler = scaler_dict["dist_region"]
    df["dist_region"] = scaler.transform(np.clip(np.log(df["dist_region"] + 1).values.reshape(-1, 1), PREPROC_PARAMS["dist_region_low"], PREPROC_PARAMS["dist_region_high"]))

    return df

In [46]:
def make_block(df, pos, radius, block_size, PREPROC_PARAMS):
    bins = [0, 1, 2, 3, 4, 5, 6, 7, 10, 14, 21, 30, 60, 180, 1e8]
    lat, lon = pos.split("_")
    lat, lon = float(lat), float(lon)
    tmp1 = df[df["pos"] == pos]
    diff = int(radius / 111) + 3
    tmp2 = df[((df["latitude"] >= lat - diff) & (df["latitude"] <= lat + diff) & (df["longitude"] >= lon - diff) & (df["longitude"] <= lon + diff)) & (df["pos"] != pos)]
    tmp1["label"] = 0
    tmp2["label"] = -1
    tmp = pd.concat([tmp1, tmp2], axis=0)
    tmp = tmp[tmp["distance"] <= radius]
    tmp.sort_values(by=["time"], inplace=True)
    tmp["diff_days"] = (tmp["time"] - tmp["time"].shift(1)).dt.days
    tmp.dropna(inplace=True)
    tmp["diff_days"] = np.digitize(tmp["diff_days"], bins=bins) - 1
    for idx in range(1, block_size):
        tmp["mag_" + str(idx)] = tmp["mag"].shift(idx)
        tmp["depth_" + str(idx)] = tmp["depth"].shift(idx)
        tmp["latitude_new_" + str(idx)] = tmp["latitude_new"].shift(idx)
        tmp["longitude_new_" + str(idx)] = tmp["longitude_new"].shift(idx)
        tmp["dist_" + str(idx)] = tmp["dist"].shift(idx)
        tmp["distance_" + str(idx)] = tmp["distance"].shift(idx) / PREPROC_PARAMS["scale_distance_lag"]
        tmp["plate_" + str(idx)] = tmp["plate"].shift(idx)
        tmp["diff_days_" + str(idx)] = tmp["diff_days"].shift(idx)
        tmp["magType_" + str(idx)] = tmp["magType"].shift(idx)
    tmp = tmp[tmp["label"] != -1]
    tmp["distance"] = tmp["distance"] / PREPROC_PARAMS["scale_distance"]
    tmp.dropna(inplace=True)
    return tmp

In [25]:
def reshape(df, block_size, feature_order, featrues_region):
    x_ts = df[feature_order].to_numpy().reshape(-1, block_size, len(feature_order) // block_size)
    x_region = df[featrues_region].to_numpy().reshape(-1, len(featrues_region))
    return x_ts, x_region

In [26]:
features_region = ["lat_cent", "lon_cent", "dist_region", "plate_region"]
featrues = ["mag", "depth", "latitude_new", "longitude_new", "dist", "distance", "plate", "diff_days", "magType"]
featrues_order = [featrues[idx] + "_" + str(i) for i in range(BLOCK_SIZE-1, 0, -1) for idx in range(len(featrues))]
featrues_order = featrues_order + featrues

In [34]:
def make_timeseries(df, X, Y, radius, block_size, features_order, features_region, PREPROC_PARAMS, scaler_dict):
    df["time"] = pd.to_datetime(df["time"], format="mixed")
    df.sort_values(by="time", inplace=True)
    pos = str(Y - GEO_SPLIT / 2) + "_" + str(X - GEO_SPLIT / 2)
    df = preprocess_df(df, PREPROC_PARAMS, scaler_dict)
    df_pos = make_block(df, pos, radius, block_size, PREPROC_PARAMS)
    df_pos = df_pos.iloc[-1]
    x_ts, x_region = reshape(df_pos, block_size, features_order, features_region)
    return x_ts, x_region

In [28]:
scalers = pickle.load(open("../data/scalers_for_npys.pkl", "rb"))

In [47]:
df_pos = make_timeseries(df.copy(deep=True), X, Y, 300, BLOCK_SIZE, featrues_order, features_region, PREPROC_PARAMS, scalers)
df_pos

,time,longitude,latitude,depth,mag,magType,longitude_disc,latitude_disc,pos,distance,...,magType_62,mag_63,depth_63,latitude_new_63,longitude_new_63,dist_63,distance_63,plate_63,diff_days_63,magType_63
56,2023-08-01 23:52:46.899,26.2338,34.9358,0.383625,0.6625,3,25.314,34.873,34.873_25.314,0.787936,...,3.0,0.6875,0.131979,0.697612,0.572674,0.530661,0.107473,16.0,4.0,3.0
48,2023-08-29 00:10:19.406,25.4996,35.7724,0.160780,0.6875,3,25.314,34.873,34.873_25.314,0.672291,...,13.0,0.6750,0.411250,0.692190,0.573437,0.289154,0.411549,16.0,1.0,3.0


In [45]:
X, Y

(25.814, 35.373)

In [33]:
df_pos[0].shape, df_pos[1].shape

((1, 64, 9), (1, 4))

In [36]:
model.predict(df_pos[0].astype(np.float32))

2024-01-03 08:06:49.308725: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8904
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /home/majkel/miniconda3/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget


1/1 [==============================] - 6s 6s/step


array([[0.05832909]], dtype=float32)

In [1]:
import tensorflow as tf

2024-01-03 08:20:29.147019: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-03 08:20:29.147176: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-03 08:20:29.147522: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-03 08:20:29.204845: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-03 08:20:30.935282: W tensorflow/compiler/

In [18]:
model = tf.keras.models.load_model("../models/model_v2_cw.keras") 

TypeError: Could not locate class 'Transformer'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': None, 'class_name': 'Transformer', 'config': {'num_layers': 2, 'd_model': 72, 'num_heads': 2}, 'registered_name': 'Custom>Transformer', 'build_config': {'input_shape': [[None, 64, 9], [None, 4]]}}